In [47]:
'''%pip install tensorflow==2.2.0
%pip install tensorflow-gpu==2.2.0 '''
#%pip install dicom2nifti
import os
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import random
import dicom2nifti
import dicom2nifti.settings as settings
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, StratifiedShuffleSplit
import cv2
import nibabel as nib    #para leer los nifti
from scipy import ndimage   #Multidimensional image processing

import matplotlib.pyplot as plt
import cv2

############################################# 
#.........DEFINICIÓN DE LOS CSV..............
#############################################
def crear_csv(train_list, label_list, name_csv, data=[]):
    if name_csv is not 'TRAIN':
        train_list=np.transpose(train_list)
        label_list=np.transpose(label_list)

        data = {'name': train_list,
                'label': label_list}

    df = pd.DataFrame(data, columns = ['name', 'label']) 
    df.to_csv('/srv/data/HIC/datos/csv/NIFTI_'+name_csv+'.csv') 
    
############################################# 
#...............PREPROCESADO.................
#############################################

def read_nifti_file(filepath):   #Aquí solo define la función. 
    """Read and load volume"""
    # Read file
    #print(filepath)
    scan = nib.load(filepath)   #nib es una clase 
    #print(scan.dataobj.slope, scan.dataobj.inter)
    #print(scan.shape)
    # Get raw data
    scan = scan.get_fdata()
    #sample_stack(scan, rows=4, cols=4, start_with=0, show_every=1)   
    n,m,l=scan.shape
    img_data=np.zeros([128,128,l])
    for i in range(0,l):
        img_data[:,:,i]=cv2.resize(scan[:,:,i],(128,128))
    #hist(img_data)
     
    #print(scan[0,0,0])
    
    
    
    return img_data



def normalize(image):
    MIN_BOUND=15
    MAX_BOUND=100
    
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    
    image[image>1] = 0. #quitar información
    image[image<0] = 0.
    image = ndimage.rotate(image, 90, reshape=False)
    return image


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 45
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

def resize_volume_black(img):  
    n,m,l=img.shape
    #print('img',img.shape)
    img_data=np.zeros([l,128,128])
    for i in range(0,l):
        img_data[i,:,:]=cv2.resize(img[:,:,i],(128,128))
    img_p=list(img_data)
    
    
    n,m,l=img_data.shape
    matriz_float = np.zeros((m,l))
    #print('matriz_float',matriz_float.shape)
    
    matriz_float = np.float32(matriz_float)
    for ind in range(n,45):
        img_p.append(matriz_float)
    img_p=np.array(img_p)
    #print('img_nuevo',img_p.shape)
    #sample_stack(img_p, rows=6, cols=6, start_with=0, show_every=1)
    
    n,m,l=img_p.shape
    img_data=np.zeros([128,128,n])
    
    for i in range(0,n):
        img_data[:,:,i]=img_p[i,:,:]
    
    return img_data

#COMPENDIO DE LAS FUNCIONES DEFINIDAS ARRIBA:
def process_scan(path,direc):
    """Read and resize volume"""
    #print(path)
    #name_image=os.listdir(path)
    #path_image= os.path.join(path, name_image[0])
    #print(path[-3:-1])
   
    # Read scan
    volume = read_nifti_file(path)    #no importan tanto el nombre (path o filepath) sino el ORDEN!!!
    
    #hist(volume)
    #sample_stack(volume, rows=4, cols=4, start_with=0, show_every=1)
    
    # Normalize
    volume = normalize(volume)      #da igual que ponga volume o pepito, lo importante es que sea la IMAGEN!
    #hist(volume)
    #sample_stack(volume, rows=4, cols=4, start_with=0, show_every=1)
    
    # Resize width, height and depth
    volume = resize_volume_black(volume)
    #hist(volume)
    #sample_stack(volume, rows=6, cols=6, start_with=0, show_every=1)
    np.save('/srv/data/HIC/datos/NIFTI_PREPROCESADO/'+str(direc)+'.npy',volume)   #./ = la dirección desde donde tengo yo este archivo si está en la misma carpeta que el notebook. si dicomtonifti está dentro de la carpeta code
    return volume

# VISUALIZAR CORTES
def sample_stack(stack, rows=5, cols=5, start_with=0, show_every=1):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[:,:,ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()
    
# VISUALIZAR CORTES
def hist(stack):
    fig= plt.hist(stack.flatten(), bins=80)
    plt.xlabel("Hounsfield Units (HU)")
    plt.ylabel("Frequency")
    plt.show()

In [41]:
'''############################################# 
#...............DICOM TO NIFTI...............
#############################################
#HIC
print('[INFO] CAMBIO DE FORMATO HIC DICOM TO NIFTI')
for dirpath, dirnames, fileList in os.walk('/srv/data/HIC/datos/HIC/DATASETS_HIC/'):
    for subfolder in dirnames:
        print(subfolder)
        dicom_directory='/srv/data/HIC/datos/HIC/DATASETS_HIC/'+subfolder
        output_folder='/srv/data/HIC/datos/NIFTI/HIC/'+subfolder
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        settings.disable_validate_orthogonal() #importante que esté
        settings.disable_validate_slice_increment() #importante que esté
        settings.disable_validate_orientation() #importante que esté
        dicom2nifti.convert_directory(dicom_directory, output_folder)


# Controles
import pandas as pd
print('[INFO] CAMBIO DE FORMATO CONTROLES DICOM TO NIFTI')
for dirpath, dirnames, fileList in os.walk('/srv/data/HIC/datos/CONTROL/DICOM_NORMALES/CONTROLES_DICOM_CODIFICADOS/FILESET/'):
    for subfolder in dirnames:
        print(subfolder)
        dicom_directory='/srv/data/HIC/datos/CONTROL/DICOM_NORMALES/CONTROLES_DICOM_CODIFICADOS/FILESET/'+subfolder+'/0/0/'
        output_folder='/srv/data/HIC/datos/NIFTI/CONTROL/'+subfolder
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        settings.disable_validate_orthogonal() #importante que esté
        settings.disable_validate_slice_increment() #importante que esté
        settings.disable_validate_orientation()
        dicom2nifti.convert_directory(dicom_directory, output_folder)
print('[INFO] FIN DEL CAMBIO DE FORMATO CONTROLES DICOM TO NIFTI')'''

"############################################# \n#...............DICOM TO NIFTI...............\n#############################################\n#HIC\nprint('[INFO] CAMBIO DE FORMATO HIC DICOM TO NIFTI')\nfor dirpath, dirnames, fileList in os.walk('/srv/data/HIC/datos/HIC/DATASETS_HIC/'):\n    for subfolder in dirnames:\n        print(subfolder)\n        dicom_directory='/srv/data/HIC/datos/HIC/DATASETS_HIC/'+subfolder\n        output_folder='/srv/data/HIC/datos/NIFTI/HIC/'+subfolder\n        if not os.path.exists(output_folder):\n            os.makedirs(output_folder)\n        settings.disable_validate_orthogonal() #importante que esté\n        settings.disable_validate_slice_increment() #importante que esté\n        settings.disable_validate_orientation() #importante que esté\n        dicom2nifti.convert_directory(dicom_directory, output_folder)\n\n\n# Controles\nimport pandas as pd\nprint('[INFO] CAMBIO DE FORMATO CONTROLES DICOM TO NIFTI')\nfor dirpath, dirnames, fileList in os.walk(

In [31]:
SPLIT = 5  
test_split=0.20

In [46]:


############################################# 
#............CONOCER QUÉ TENEMOS.............
#############################################

# Folder "CONTROL" consist of CT scans having normal brain tissue.
normal_scan_paths = [
    os.path.join("/srv/data/HIC/datos/NIFTI/CONTROL/", x)      #está juntando la ruta donde está el colab con Mosmed... con x 
    for x in os.listdir("/srv/data/HIC/datos/NIFTI/CONTROL/")     #os.listdir () --> te hace una lista de lo que hay dentro de la carpeta que pongas ()
]

# Folder "CONTROL" consist of CT scans having abnormal brain tissue.
abnormal_scan_paths = [
    os.path.join("/srv/data/HIC/datos/NIFTI/HIC/", x)
    for x in os.listdir("/srv/data/HIC/datos/NIFTI/HIC/")
]

print("CT scans with normal brain tissue: " + str(len(normal_scan_paths)))
print("CT scans with abnormal brain tissue: " + str(len(abnormal_scan_paths)))


############################################# 
#..........CREAMOS LOS CSV...................
#############################################
   
#CREAMOS EL EXCEL CON LOS PATHS Y LAS ETIQUETAS: 
tipo=['HIC','CONTROL']
for itipo in tipo:
    train_list=[]
    label_list = []
    for dirpath, dirnames, fileList in os.walk('/srv/data/HIC/datos/NIFTI/'+str(itipo)+'/'):
        for direc in dirnames:
            print(direc)
            for dirpath1, dirnames1, fileList1 in os.walk('/srv/data/HIC/datos/NIFTI/'+str(itipo)+'/'+str(direc)+'/'):
                file=fileList1[0]
                process_scan(dirpath1+file,direc)
            
        
            train_list.append('/srv/data/HIC/datos/NIFTI_PREPROCESADO/'+str(direc)+'.npy')
            if str(itipo) is 'HIC':
                label_list.append(1) 
            else:
                label_list.append(0) 
    
    if str(itipo) is 'CONTROL':
        train_list_CONTROL=train_list
        label_list_CONTROL=label_list
        crear_csv(train_list_CONTROL, label_list_CONTROL, 'TRAIN_CONTROL')
    else:
        train_list_HIC=train_list
        label_list_HIC=label_list 
        crear_csv(train_list_HIC, label_list_HIC, 'TRAIN_HIC')

train_list_CONTROL=np.transpose(train_list_CONTROL)
label_list_CONTROL=np.transpose(label_list_CONTROL)
train_list_HIC=np.transpose(train_list_HIC)
label_list_HIC=np.transpose(label_list_HIC)

names=np.concatenate((train_list_CONTROL, train_list_HIC))
labels=np.concatenate((label_list_CONTROL, label_list_HIC))

#TRAIN
data = {'name': names,'label': labels}
crear_csv(names, labels, '_', data)

CT scans with normal brain tissue: 96
CT scans with abnormal brain tissue: 118
99
93
61
259
230
149
117
135
235
21
98
96
90
282
7
286
288
34
37
141
87
15
256
268
285
42
250
124
237
266
51
86
127
119
125
292
131
132
147
232
53
108
290
283
271
75
82
13
240
95
281
28
12
63
246
88
74
69
54
35
274
261
148
122
234
10
30
39
29
239
77
264
14
26
249
62
106
118
144
120
102
50
110
128
8
126
27
284
257
277
36
243
100
244
115
245
236
20
66
92
121
3
79
24
17
272
146
68
242
112
104
238
91
296
78
269
70
97
140
31
116
99
32
23
149
117
123
135
150
21
98
7
33
0
101
103
105
37
141
15
107
114
42
124
44
4
47
127
134
11
113
119
125
45
131
132
147
138
16
108
41
46
13
143
9
5
129
28
12
43
6
148
122
10
18
30
145
1
19
39
2
29
142
14
26
106
118
139
144
120
102
50
48
110
130
128
8
109
136
126
27
36
49
100
115
20
40
121
3
24
17
146
112
137


In [48]:
############################################# 
#.............CREAR KFOLDS...................
#############################################

# Read data
# Assuming it has two cols:
# name: path to each image with its extension
# label: labels (here it is 0s and 1s) -> binary classification
df = pd.read_csv('/srv/data/HIC/datos/csv/NIFTI__.csv')
df = df.iloc[np.random.permutation(len(df))]
df = df.astype(str)

train_df, test_df = train_test_split(df, test_size=0.25)

print('[INFO] TRAIN CSV ',train_df.shape)
print('[INFO] TEST CSV ',test_df.shape)
train_df.to_csv('/srv/data/HIC/datos/csv/NIFTI_TRAIN.csv')
test_df.to_csv('/srv/data/HIC/datos/csv/NIFTI_TEST.csv')

Xtrain = train_df['name']
ytrain = train_df['label']

print("KFold, shuffle=False (default)")
kf = KFold(n_splits=SPLIT, random_state=True, shuffle=True)

X_train=[]
X_val=[]
y_train=[]
y_val=[]
i=1
for train_index, test_index in kf.split(Xtrain, ytrain):
    train_index=list(train_index)
    test_index=list(test_index)

    Xtrain = np.array(Xtrain)
    ytrain = np.array(ytrain)
    if(i is 1):
        X_train1 , X_val1 = Xtrain[train_index], Xtrain[test_index]
        y_train1, y_val1 = ytrain[train_index], ytrain[test_index]
        
        data = {'name': X_train1,
        'label': y_train1}
        dftrain1 = pd.DataFrame(data, columns = ['name', 'label']) 
        dftrain1.to_csv('/srv/data/HIC/datos/csv/NIFTI_TRAIN'+str(i)+'.csv')
        
        data = {'name': X_val1,
        'label': y_val1}
        dfval1 = pd.DataFrame(data, columns = ['name', 'label']) 
        dfval1.to_csv('/srv/data/HIC/datos/csv/NIFTI_VAL'+str(i)+'.csv')
        
        i=i+1
    elif(i is 2):
        X_train2 , X_val2 = Xtrain[train_index], Xtrain[test_index]
        y_train2, y_val2 = ytrain[train_index], ytrain[test_index]
        
        data = {'name': X_train2,
        'label': y_train2}
        dftrain2 = pd.DataFrame(data, columns = ['name', 'label']) 
        dftrain2.to_csv('/srv/data/HIC/datos/csv/NIFTI_TRAIN'+str(i)+'.csv')
        
        data = {'name': X_val2,
        'label': y_val2}
        dfval2 = pd.DataFrame(data, columns = ['name', 'label']) 
        dfval2.to_csv('/srv/data/HIC/datos/csv/NIFTI_VAL'+str(i)+'.csv')
        
        i=i+1
    elif(i is 3):
        X_train3 , X_val3 = Xtrain[train_index], Xtrain[test_index]
        y_train3, y_val3 = ytrain[train_index], ytrain[test_index]
        
        data = {'name': X_train3,
        'label': y_train3}
        dftrain3 = pd.DataFrame(data, columns = ['name', 'label']) 
        dftrain3.to_csv('/srv/data/HIC/datos/csv/NIFTI_TRAIN'+str(i)+'.csv')
        
        data = {'name': X_val3,
        'label': y_val3}
        dfval3 = pd.DataFrame(data, columns = ['name', 'label']) 
        dfval3.to_csv('/srv/data/HIC/datos/csv/NIFTI_VAL'+str(i)+'.csv')
        
        i=i+1
    elif(i is 4):
        X_train4 , X_val4 = Xtrain[train_index], Xtrain[test_index]
        y_train4, y_val4 = ytrain[train_index], ytrain[test_index]
        
        data = {'name': X_train4,
        'label': y_train4}
        dftrain4 = pd.DataFrame(data, columns = ['name', 'label']) 
        dftrain4.to_csv('/srv/data/HIC/datos/csv/NIFTI_TRAIN'+str(i)+'.csv')
        
        data = {'name': X_val4,
        'label': y_val4}
        dfval4 = pd.DataFrame(data, columns = ['name', 'label']) 
        dfval4.to_csv('/srv/data/HIC/datos/csv/NIFTI_VAL'+str(i)+'.csv')
        
        i=i+1
    elif(i is 5):
        X_train5 , X_val5 = Xtrain[train_index], Xtrain[test_index]
        y_train5, y_val5 = ytrain[train_index], ytrain[test_index]
        
        data = {'name': X_train5,
        'label': y_train5}
        dftrain5 = pd.DataFrame(data, columns = ['name', 'label']) 
        dftrain5.to_csv('/srv/data/HIC/datos/csv/NIFTI_TRAIN'+str(i)+'.csv')
        
        data = {'name': X_val5,
        'label': y_val5}
        dfval5 = pd.DataFrame(data, columns = ['name', 'label']) 
        dfval5.to_csv('/srv/data/HIC/datos/csv/NIFTI_VAL'+str(i)+'.csv')      
        i=i+1
        
#REVISAR KFOLDS HECHOS Y TAMAÑO   
print('TRAIN 1',dftrain1.shape)
print('VAL 1', dfval1.shape)
print('\n')

print('TRAIN 2',dftrain2.shape)
print('VAL 2',dfval2.shape)
print('\n')

print('TRAIN 3',dftrain3.shape)
print('VAL 3',dfval3.shape)
print('\n')

print('TRAIN 4',dftrain4.shape)
print('VAL 4',dfval4.shape)
print('\n')

print('TRAIN 5',dftrain5.shape)
print('VAL 5',dfval5.shape)
print('\n')

[INFO] TRAIN CSV  (160, 3)
[INFO] TEST CSV  (54, 3)
KFold, shuffle=False (default)
TRAIN 1 (128, 2)
VAL 1 (32, 2)


TRAIN 2 (128, 2)
VAL 2 (32, 2)


TRAIN 3 (128, 2)
VAL 3 (32, 2)


TRAIN 4 (128, 2)
VAL 4 (32, 2)


TRAIN 5 (128, 2)
VAL 5 (32, 2)




In [27]:
x_train=[]
y_train=[]
x_val=[]
y_val=[]
CSV_TRAIN=[]
CSV_VAL=[]
    
print('[INFO] LEYENDO CSV \n')

CSV_TRAIN = pd.read_csv('/srv/data/HIC/datos/csv/NIFTI_TRAIN.csv',sep=',')
CSV_VAL = pd.read_csv('/srv/data/HIC/datos/csv/NIFTI_TEST.csv',sep=',')
train_list=[]
label_list=[]
for path in CSV_TRAIN['name']:
    print(path)
    process_scan(path) #es un array 4D (100 (nº de TC) x 128 x 128 x 64)
    train_list.append('/srv/data/HIC/datos/NIFTI_PREPROCESADO/'+str(path[-3:-1])+'.npy')

print(train_list_HIC, CSV_TRAIN['name'])

#Crear CSV train
data = {'name': train_list,'label': CSV_VAL['label']}
crear_csv(names, labels, 'TRAIN', data)


train_list=[]
label_list=[]
for path in CSV_VAL['name']:
    print(path)
    process_scan(path) #es un array 4D (100 (nº de TC) x 128 x 128 x 64)
    train_list.append('/srv/data/HIC/datos/NIFTI_PREPROCESADO/'+str(path[-3:-1])+'.npy')

print(train_list_HIC, CSV_VAL['name'])
  

#Crear CSV train
#TRAIN
data = {'name': train_list,'label': CSV_VAL['label']}
crear_csv(names, labels, 'TRAIN', data)


x_val = [process_scan(path) for path in CSV_VAL['name']]
y_val=CSV_VAL['label']

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_val = np.asarray(x_val)
y_val = np.asarray(y_val)

print("Number of samples in train and validation are %d and %d." % (x_train.shape[0], x_val.shape[0]))

[INFO] LEYENDO CSV 

/srv/data/HIC/datos/NIFTI/CONTROL/40/
/srv/data/HIC/datos/NIFTI/CONTROL/24/
/srv/data/HIC/datos/NIFTI/CONTROL/99/
/srv/data/HIC/datos/NIFTI/CONTROL/131/
/srv/data/HIC/datos/NIFTI/HIC/121/
/srv/data/HIC/datos/NIFTI/CONTROL/29/
/srv/data/HIC/datos/NIFTI/HIC/82/
/srv/data/HIC/datos/NIFTI/HIC/10/
/srv/data/HIC/datos/NIFTI/HIC/115/
/srv/data/HIC/datos/NIFTI/HIC/15/
/srv/data/HIC/datos/NIFTI/CONTROL/10/
/srv/data/HIC/datos/NIFTI/CONTROL/124/
/srv/data/HIC/datos/NIFTI/HIC/268/
/srv/data/HIC/datos/NIFTI/HIC/257/
/srv/data/HIC/datos/NIFTI/CONTROL/109/
/srv/data/HIC/datos/NIFTI/HIC/42/
/srv/data/HIC/datos/NIFTI/HIC/62/
/srv/data/HIC/datos/NIFTI/HIC/269/
/srv/data/HIC/datos/NIFTI/CONTROL/32/
/srv/data/HIC/datos/NIFTI/HIC/264/
/srv/data/HIC/datos/NIFTI/CONTROL/148/
/srv/data/HIC/datos/NIFTI/HIC/148/
/srv/data/HIC/datos/NIFTI/HIC/88/
/srv/data/HIC/datos/NIFTI/CONTROL/30/
/srv/data/HIC/datos/NIFTI/CONTROL/145/
/srv/data/HIC/datos/NIFTI/CONTROL/0/
/srv/data/HIC/datos/NIFTI/CONTRO

FileNotFoundError: [Errno 2] No such file or directory: '/srv/data/HIC/datos/NIFTI/HIC/93/'

In [ ]:
tip='HIC'
paciente=93
import nibabel as nib
import os
# VISUALIZAR CORTES
def sample_stack(stack, rows=5, cols=5, start_with=0, show_every=1):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[:,:,ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()
    
j= os.listdir('/srv/data/HIC/datos/NIFTI/'+tip+'/'+str(paciente))
print(j)
img = nib.load('/srv/data/HIC/datos/NIFTI/'+tip+'/'+str(paciente)+'/'+j[0])
image_data = img.get_fdata()#
print(image_data.shape)
sample_stack(image_data, rows=5, cols=5, start_with=0, show_every=1)
'''    
import matplotlib.pyplot as plt
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.imshow(image_data[:,:,27],cmap='gray')'''